<a href="https://colab.research.google.com/github/ueki5/colaboratory/blob/main/playground-009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 必要ライブラリ・コマンドの導入
!pip install japanize_matplotlib | tail -n 1
!pip install torchviz | tail -n 1
w = !apt install tree
print(w[-2])

Setting up tree (2.0.2-1) ...


In [1]:
import torch
# レイヤー関数定義

# 最初の線形関数
# 784 入力数
# 128 出力数
l1 = torch.nn.Linear(784, 128)

# ２番目の線形関数
# 128 入力数
#  10 出力数
l2 = torch.nn.Linear(128, 10)

# 活性化関数
relu = torch.nn.ReLU(inplace=True)




In [4]:
# 入力テンソルから出力テンソルを計算

# ダミー入力データを作成
inputs = torch.randn(100, 784)

# 中間テンソル1の計算
m1 = l1(inputs)

# 中間テンソル2の計算
m2 = relu(m1)

# 出力テンソルの計算
outputs = l2(m2)

# 入力テンソルと出力テンソルのshape確認
print(f'入力テンソル:{inputs.shape}')
print(f'出力テンソル:{outputs.shape}')

# 入力テンソルと出力テンソル確認
print(f'入力テンソル:{inputs}')
print(f'出力テンソル:{outputs}')



入力テンソル:torch.Size([100, 784])
出力テンソル:torch.Size([100, 10])
